In [47]:
import sys
import cv2 as cv
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import Delaunay
from scipy.signal import convolve2d
from PIL import Image, ImageOps, ImageDraw

In [48]:
#Functions---------------------------------

def plot(img):
    plt.imshow(img, cmap = 'gray')
    plt.show()

def pad_img(img1, img2):
    m1, n1 = img1.shape[:2]
    m2, n2 = img2.shape[:2]
    x_pad = np.abs(m2-m1)/2
    y_pad = np.abs(n2-n1)/2
    img2 = img2[x_pad:m1+x_pad, y_pad:n1+y_pad]
    return img2

In [69]:
#Variables----------------------------------

ddepth = cv.CV_16S
kernel_size = 3
n=10000

In [63]:
#1) Load Image-------------------------------------
img_name = 'beatles_-_abbey_road.jpg'
src = cv.imread(img_name, cv.IMREAD_COLOR)
src = cv.GaussianBlur(src, (5, 5), 0) #Remove noise by blurring
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY) #Convert to Grayscale


width = src.shape[0]
height = src.shape[1]

temp = Image.open(img_name)
img_np = temp.load()

In [70]:
#2) Canny Edge Detection----------------------------
lower = 100
upper = 250
edges = cv.Canny(src_gray, lower, upper)
edges_new = cv.cvtColor(edges, cv.COLOR_GRAY2RGB)
#mask = cv.inRange(edges_new, np.array([255,255,255]), np.array([255,255,255]))
#res = cv.bitwise_and(edges_new, edges_new, mask=mask)
#cv.imshow("", edges_new)
#cv.waitKey(0)
#cv.destroyAllWindows()

In [120]:

def search_coordinate(df_data: pd.DataFrame, search_set: set) -> list:
    nda_values = df_data.values
    tuple_index = np.where(np.isin(nda_values, [e for e in search_set]))
    return [(col, row) for row, col in zip(tuple_index[0], tuple_index[1])]

sample = pd.DataFrame(data = edges)
searches= search_coordinate(sample, {255})
coords = random.choices(searches, k = n//4)

In [121]:
pic = Image.new(mode = "RGB", size=(width, height))
draw = ImageDraw.Draw(pic)
draw.point(coords)
#pic.show()

tri = Delaunay(coords)
#test = pd.DataFrame(data = coords)
#draw.polygon(tri.simplices)
new_coords = np.array(coords)
#new_coords[tri.simplices]
#i=0
for triangle in new_coords[tri.simplices]:
    row = triangle[1].tolist()[0]
    col = triangle[1].tolist()[1]
    color = img_np[col, row]
    draw.polygon([(triangle[0].tolist()[1], triangle[0].tolist()[0]), (triangle[1].tolist()[1], triangle[1].tolist()[0]), (triangle[2].tolist()[1], triangle[2].tolist()[0])], fill=color)
    #i += 1
pic.show()

In [82]:
sample1=pd.DataFrame(data=edges)
sample2=pd.DataFrame(data=edges)
max_index = sample1.append(sample2)
#max_index = pd.DataFrame(data=edges).stack().sort_values(ascending=False).index[0:n]

x = max_index.get_level_values(1)
y = max_index.get_level_values(0)

coords = [(i, j) for i, j in zip(x, y)]
coords_reverse = [(j, i) for i, j in zip(x, y)]
coord_list = [[i, j] for i, j in zip(x, y)]
#triangulate_coords = [(i, height-j) for i, j in zip(x, y)]

#chance_array = pd.DataFrame(data=chances)
#chance_array.hood = 
#unique_vals = pd.DataFrame(data=chances).stack().unique()
#test
#PIL indexes [col, row], or [y, x] Delaunay indexes [x, y], starting from 0

#Need to figure out how to unpack it to make it triangulate the correct way

#3) Create new PIL image and plot points

pic = Image.new(mode = "RGB", size=(width, height))
draw = ImageDraw.Draw(pic)
draw.point(coords)
pic.show()

tri = Delaunay(coords)
test = pd.DataFrame(data = coords)
#draw.polygon(tri.simplices)
new_coords = np.array(coords)
#new_coords[tri.simplices]
#i=0
for triangle in new_coords[tri.simplices]:
    row = triangle[1].tolist()[0]
    col = triangle[1].tolist()[1]
    color = img_np[col, row]
    draw.polygon([(triangle[0].tolist()[1], triangle[0].tolist()[0]), (triangle[1].tolist()[1], triangle[1].tolist()[0]), (triangle[2].tolist()[1], triangle[2].tolist()[0])], fill=color)
    #i += 1
#pic.show()

IndexError: Too many levels: Index has only 1 level, not 2

In [35]:
#2) Transform Image



#Sobel--------------------------------------
sobel_x = cv.Sobel(src_gray, cv.CV_32F, 1, 0, ksize = 3)
sobel_y = cv.Sobel(src_gray, cv.CV_32F, 0, 1, ksize = 3)
sobel_first = cv.magnitude(sobel_x, sobel_y)

sobel = np.empty_like(sobel_first)
sobel[:] = sobel_first
sobel[sobel > 50] = 255
sobel[sobel < 50] = 0
#sobel_gray = cv.cvtColor(sobel, cv.COLOR_BGR2GRAY)

#Laplacian-----------------------------------

LoG = cv.Laplacian(src_gray, ddepth, ksize = 3)
minLoG = cv.morphologyEx(LoG, cv.MORPH_ERODE, np.ones((3, 3)))
maxLoG = cv.morphologyEx(LoG, cv.MORPH_DILATE, np.ones((3, 3)))
zeroCross = np.logical_or(np.logical_and(minLoG < 0, LoG > 0), np.logical_and(maxLoG > 0, LoG < 0))
 


edges_df = pd.DataFrame(data = LoG)
zeroCross_df = pd.DataFrame(data = zeroCross)

#cv.imshow("", cv.bitwise_and(zeroCross, sobel))
#cv.waitKey(0)
#cv.destroyAllWindows()

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'bitwise_and'
> Overload resolution failed:
>  - src1 data type = 0 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src1'
